In [54]:
from warnings import simplefilter, filterwarnings
from sklearn.exceptions import ConvergenceWarning
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)

simplefilter(action='ignore', category=FutureWarning)
filterwarnings('ignore')

# Menu
<a name="navegacao"></a>

## 1) [Data cleaning and feature engineering](#cleaning)
- 1.1 [Encoding de Conglomerado e Natureza Jurídica](#encoding)
- 1.2 [Descrição dos dados](#descricao)
- 1.3 [Feature engineering](#engineering)
- 1.4 [Train/Test partition](#partition)



## 2) [Random Search for Hyperparameters Tunning](#randomsearch)
- 2.1 [Suport Vector Machine](#svm1)
- 2.2 [Stochastic Gradient Descent](#sgd1)
- 2.3 [Multi-Task Elastic-Net](#elastic1)
- 2.4 [Least Angle Regression Model (LAR)](#LAR1)
- 2.5 [Ridge Regression](#ridge1)
- 2.6 [ElasticNet Regression](#ElasNet1)
- 2.7 [Bayesian Ridge](#bayesian1)
- 2.8 [Lasso Regression](#lasso1)
- 2.9 [Multilayer Perceptron Regression](#mlp1)
- 2.10 [Nearest Neighbors Regression](#nnr1)
- 2.11 [Resumo dos resultados](#resumo_rs1)


# 3) [Random Halving Serach for Hyperparameters Tunning](#halving)
- 3.1 [Suport Vector Machine](#svm2)
- 3.2 [Stochastic Gradient Descent](#sgd2)
- 3.3 [Multi-Task Elastic-Net](#elastic2)
- 3.4 [Least Angle Regression Model (LAR)](#LAR2)
- 3.5 [Ridge Regression](#ridge2)
- 3.6 [ElasticNet Regression](#ElasNet2)
- 3.7 [Bayesian Ridge](#bayesian2)
- 3.8 [Lasso Regression](#lasso2)
- 3.9 [Multilayer Perceptron Regression](#mlp2)
- 3.10 [Nearest Neighbors Regression](#nnr2)
- 3.11 [Resumo dos resultados](#resumo_rs2)

In [55]:
from time import time
import requests
import pandas as pd
import numpy as np 
import random 
import os
import pickle
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost

In [56]:
%%time

spread = pd.read_sas('dados bndes/dadospreadpy.sas7bdat' , encoding='latin-1')
#https://www.analyticsvidhya.com/blog/2018/03/introduction-regression-splines-python-codes/

Wall time: 9.67 s


### Veja tutorial sobre lags em um painel de dados:

- https://towardsdatascience.com/timeseries-data-munging-lagging-variables-that-are-distributed-across-multiple-groups-86e0a038460c

### Veja tutorial sobre Bayesian Hyperparameter Tunning for ML:

- https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f

In [57]:
#spread.columns

<a name="cleaning"></a>


# 1) Data Cleaning and Feature Engineering

- ir para [Menu Principal](#navegacao)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Train/Test partition](#partition)


<a name="encoding"></a>

## 1.1) Encoding de Conglomerado e Natureza Jurídica


In [58]:
%%time

spread.fillna(0,inplace=True)

Wall time: 106 ms


In [59]:
%%time

enc_congl = spread[['CNPJ','Conglomerado']].groupby(['Conglomerado']).count().reset_index().rename(columns={"CNPJ":"banco_enc"})
enc_congl.head()

Wall time: 116 ms


,Conglomerado,banco_enc
0,0,23462
1,ABC-BRASIL,289
2,ALFA,435
3,BANCOOB,389
4,BANESTES,34


In [60]:
%%time
spread = pd.merge( spread, enc_congl, left_on=['Conglomerado'],right_on=['Conglomerado'], how='inner')

Wall time: 173 ms


In [61]:
%%time

enc_natjur = spread[['CNPJ','NATJUR']].groupby(['NATJUR']).count().reset_index().rename(columns={"CNPJ":"firma_enc"})
enc_natjur.head()

Wall time: 93.9 ms


,NATJUR,firma_enc
0,1015,6
1,1031,65
2,1112,8
3,1244,2
4,2011,28


In [62]:
%%time
spread = pd.merge( spread, enc_natjur, left_on=['NATJUR'],right_on=['NATJUR'], how='inner')

Wall time: 134 ms


In [63]:
%%time
spread.loc[ (spread['TCB']=='b1'), 'banco_b1'] = 1
spread.loc[~(spread['TCB']=='b1'), 'banco_b1'] = 0

spread.loc[ (spread['TCB']=='b2'), 'banco_b2'] = 1
spread.loc[~(spread['TCB']=='b2'), 'banco_b2'] = 0

spread.loc[ (spread['TCB']=='b3C'), 'banco_b3'] = 1
spread.loc[~(spread['TCB']=='b3C'), 'banco_b3'] = 0

spread[['CNPJ','EMPRESA','banco_b1']].groupby(['banco_b1']).count()

,CNPJ,EMPRESA
banco_b1,,
0.0,24450,24450
1.0,34647,34647


In [64]:
%%time
spread[['CNPJ','EMPRESA','banco_b3']].groupby(['banco_b3']).count()

,CNPJ,EMPRESA
banco_b3,,
0.0,58868,58868
1.0,229,229


In [65]:
%%time
spread[['CNPJ','EMPRESA','TCB']].groupby(['TCB']).count()

,CNPJ,EMPRESA
TCB,,
b1,34647,34647
b2,22218,22218
b3C,229,229
b4,1447,1447
n1,556,556


<a name="engineering"></a>

## 1.2) Feature engineering
- ir para [Menu Principal](#navegacao)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Train/Test partition](#partition)


In [67]:
%%time
spread['target'] = spread['SpreadInstituicaoFinanceira'] 

# dados da operação
spread['oper_valor']                   = np.log(spread['ValorOperacao']) 
spread['oper_spreadbndes']             = spread['SpreadBNDES'] 
spread['oper_juros']                   = spread['Juros'] 
spread['oper_amortizacao']             = np.log(spread['Prazo_Amortiza']) 
spread['oper_carencia']                = np.log(spread['prazo_carencia']+1) 
spread['oper_valor_spreadbndes']       = np.log(spread['ValorOperacao']) - spread['SpreadBNDES']
spread['oper_valor_juros']             = np.log(spread['ValorOperacao']) - spread['Juros']
spread['oper_valor_amortizacao']       = np.log(spread['ValorOperacao']) - spread['Prazo_Amortiza']
spread['oper_valor_carencia']          = np.log(spread['ValorOperacao']) - spread['prazo_carencia'] 
spread['oper_spreadbndes_juros']       = spread['SpreadBNDES']           - spread['Juros'] 
spread['oper_spreadbndes_amortizacao'] = spread['SpreadBNDES']           - spread['Prazo_Amortiza']
spread['oper_spreadbndes_carencia']    = spread['SpreadBNDES']           - spread['prazo_carencia']
spread['oper_juros_amortizacao']       = spread['Juros']                 - spread['Prazo_Amortiza']
spread['oper_juros_carencia']          = spread['Juros']                 - spread['prazo_carencia']
spread['oper_amortizacao_carencia']    = spread['Prazo_Amortiza']        - spread['prazo_carencia']

Wall time: 15 ms


In [68]:
%%time
# dados da firma
spread['firma_potot']  = np.log(spread['POTOT_FIRMA']+1) 
spread['firma_idade']  = np.log(spread['IDADE_FIRMA']+1) 
spread['firma_massa']  = np.log(spread['MSAL_FIRMA']+1) 
spread['firma_po3gr']  = np.log(spread['PO3G_FIRMA']+1) 
spread['firma_rotat']  = np.log(spread['ROT_FIRMA']+1) 
spread['firma_engen']  = np.log(spread['ENG_FIRMA']+1) 
spread['firma_share']  = np.log(spread['SHARE_FIRMA']+1) 

spread['firma_potot_idade'] = np.log(spread['POTOT_FIRMA']+1) / np.log(spread['IDADE_FIRMA']+2) 
spread['firma_potot_massa'] = np.log(spread['POTOT_FIRMA']+1) / np.log(spread['MSAL_FIRMA']+2) 
spread['firma_potot_po3gr'] = np.log(spread['POTOT_FIRMA']+1) / np.log(spread['PO3G_FIRMA']+2) 
spread['firma_idade_massa'] = np.log(spread['IDADE_FIRMA']+1) / np.log(spread['MSAL_FIRMA']+2) 
spread['firma_idade_po3gr'] = np.log(spread['IDADE_FIRMA']+1) / np.log(spread['PO3G_FIRMA']+2) 
spread['firma_idade_engen'] = np.log(spread['IDADE_FIRMA']+1) / np.log(spread['ENG_FIRMA']+2)
spread['firma_massa_po3gr'] = np.log(spread['MSAL_FIRMA']+1)  / np.log(spread['PO3G_FIRMA']+2) 


Wall time: 42 ms


In [69]:
%%time
# dados da instituição financeira
spread['banco_ativo']            =np.log(spread['Ativo_Total']) 
spread['banco_carteira']         =np.log(spread['Carteira_Credito']) 
spread['banco_passivo']          =np.log(spread['Passivo_Circulante']) 
spread['banco_captacoes']        =np.log(spread['Captacoes']+1) 
spread['banco_patrimonio']       =np.log(spread['Patrimonio_Liquido']+1) 
spread['banco_lucroliquido']     =np.log(spread['Lucro_Liquido']+1) 
spread['banco_numagencias']      =np.log(spread['Numero_Agencias']+1) 
spread['banco_numeropostos']     =np.log(spread['Numero_Postos']+1) 
spread['banco_ativo_carteira']   =np.log(spread['Ativo_Total'])        /np.log(spread['Carteira_Credito'])   
spread['banco_ativo_passivo']    =np.log(spread['Ativo_Total'])        /np.log(spread['Passivo_Circulante']) 
spread['banco_ativo_captacoes']  =np.log(spread['Ativo_Total'])        /np.log(spread['Captacoes']+1)        
spread['banco_ativo_patrimonio'] =np.log(spread['Ativo_Total'])        /np.log(spread['Patrimonio_Liquido']) 
spread['banco_carteira_passivo'] =np.log(spread['Carteira_Credito'])   /np.log(spread['Passivo_Circulante']) 

In [70]:
%%time
spread.fillna(0,inplace=True)

Wall time: 69 ms


<a name="descricao"></a>

## 1.3) Descrição dos dados
- ir para [Menu Principal](#navegacao)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Train/Test partition](#partition)

In [71]:
%%time
spread.filter(regex='firma', axis=1).describe().transpose()

Wall time: 183 ms


,count,mean,std,min,25%,50%,75%,max
firma_enc,59097.0,33605.668257,17273.775945,1.000000e+00,6909.000000,43792.000000,43792.000000,43792.000000
firma_potot,59097.0,3.373903,1.827505,4.987542e-03,1.951040,3.063391,4.617672,9.556813
firma_idade,59097.0,2.703713,0.840707,0.000000e+00,2.149434,2.761907,3.311273,4.390367
firma_massa,59097.0,10.781991,2.256507,0.000000e+00,9.172427,10.538891,12.404739,18.422188
firma_po3gr,59097.0,1.426054,1.609216,0.000000e+00,0.000000,0.747635,2.475277,9.104200
firma_rotat,59097.0,0.218042,0.103674,0.000000e+00,0.154151,0.223144,0.287682,0.405465
firma_engen,59097.0,0.240711,0.632251,0.000000e+00,0.000000,0.000000,0.000000,5.568345
firma_share,59097.0,0.001115,0.004967,5.847944e-09,0.000011,0.000062,0.000378,0.252814
firma_potot_idade,59097.0,1.203585,0.589484,1.954530e-03,0.783232,1.156933,1.551576,7.214130
firma_potot_massa,59097.0,0.296870,0.187812,2.096026e-03,0.212821,0.291238,0.376063,6.600389


In [72]:
%%time
spread.filter(regex='banco', axis=1).corr()

Wall time: 61 ms


,banco_enc,banco_b1,banco_b2,banco_b3,banco_ativo,banco_carteira,banco_passivo,banco_captacoes,banco_patrimonio,banco_lucroliquido,banco_numagencias,banco_numeropostos,banco_ativo_carteira,banco_ativo_passivo,banco_ativo_captacoes,banco_ativo_patrimonio,banco_carteira_passivo
banco_enc,1.000000,-0.532766,0.447746,0.075236,-0.516485,-0.444404,-0.516057,-0.515313,-0.550736,-0.154619,-0.527102,-0.515653,-0.395987,0.216522,0.173180,0.085373,0.490176
banco_b1,-0.532766,1.000000,-0.923967,-0.074246,0.858257,0.788242,0.854955,0.854562,0.830789,0.733292,0.909108,0.878072,0.537921,-0.294367,-0.226798,0.230219,-0.669508
banco_b2,0.447746,-0.923967,1.000000,-0.048411,-0.768582,-0.679683,-0.754886,-0.752924,-0.762391,-0.718387,-0.840101,-0.811309,-0.594165,0.096456,0.038595,-0.113488,0.638172
banco_b3,0.075236,-0.074246,-0.048411,1.000000,-0.115770,-0.251188,-0.110988,-0.119278,-0.159035,-0.067032,-0.079617,-0.065193,0.651952,-0.018025,0.089391,0.284931,-0.475047
banco_ativo,-0.516485,0.858257,-0.768582,-0.115770,1.000000,0.973998,0.998360,0.997361,0.981031,0.784405,0.959734,0.953875,0.453633,-0.385039,-0.296069,0.193882,-0.621366
banco_carteira,-0.444404,0.788242,-0.679683,-0.251188,0.973998,1.000000,0.973470,0.973936,0.952833,0.740434,0.912584,0.912680,0.249274,-0.392107,-0.320977,0.183471,-0.444345
banco_passivo,-0.516057,0.854955,-0.754886,-0.110988,0.998360,0.973470,1.000000,0.999410,0.973585,0.775316,0.954392,0.947651,0.451194,-0.437083,-0.346099,0.224979,-0.634789
banco_captacoes,-0.515313,0.854562,-0.752924,-0.119278,0.997361,0.973936,0.999410,1.000000,0.971967,0.769555,0.955277,0.948278,0.443518,-0.443408,-0.363655,0.227505,-0.630422
banco_patrimonio,-0.550736,0.830789,-0.762391,-0.159035,0.981031,0.952833,0.973585,0.971967,1.000000,0.793420,0.950461,0.950292,0.435220,-0.294014,-0.213387,0.002162,-0.580847
banco_lucroliquido,-0.154619,0.733292,-0.718387,-0.067032,0.784405,0.740434,0.775316,0.769555,0.793420,1.000000,0.755654,0.739969,0.438623,-0.182488,-0.078307,0.025071,-0.529737


In [73]:
%%time
spread.filter(regex='oper', axis=1).corr()

Wall time: 48 ms


,oper_valor,oper_spreadbndes,oper_juros,oper_amortizacao,oper_carencia,oper_valor_spreadbndes,oper_valor_juros,oper_valor_amortizacao,oper_valor_carencia,oper_spreadbndes_juros,oper_spreadbndes_amortizacao,oper_spreadbndes_carencia,oper_juros_amortizacao,oper_juros_carencia,oper_amortizacao_carencia
oper_valor,1.000000,0.001894,-0.082998,-0.017999,0.119088,0.963103,0.442593,0.077965,0.171839,0.087102,-0.006925,-0.124685,-0.021884,-0.143618,-0.028519
oper_spreadbndes,0.001894,1.000000,0.439267,-0.280726,0.078473,-0.267308,-0.394568,0.287715,-0.045006,-0.340261,0.310018,0.037314,0.341931,0.212503,-0.299802
oper_juros,-0.082998,0.439267,1.000000,-0.468580,-0.037531,-0.198199,-0.930363,0.450964,0.066276,-0.994218,0.466535,0.128063,0.600958,0.635669,-0.430714
oper_amortizacao,-0.017999,-0.280726,-0.468580,1.000000,0.098544,0.058209,0.415019,-0.946958,-0.102671,0.456919,-0.948220,-0.121443,-0.941217,-0.340641,0.914893
oper_carencia,0.119088,0.078473,-0.037531,0.098544,1.000000,0.093635,0.077585,-0.098277,-0.879206,0.048665,-0.106100,-0.914776,-0.104875,-0.735148,-0.153872
oper_valor_spreadbndes,0.963103,-0.267308,-0.198199,0.058209,0.093635,1.000000,0.532680,-0.002305,0.177699,0.175508,-0.090110,-0.130190,-0.113113,-0.195584,0.053206
oper_valor_juros,0.442593,-0.394568,-0.930363,0.415019,0.077585,0.532680,1.000000,-0.377105,0.003584,0.926670,-0.422349,-0.161107,-0.548810,-0.624831,0.377076
oper_valor_amortizacao,0.077965,0.287715,0.450964,-0.946958,-0.098277,-0.002305,-0.377105,1.000000,0.133850,-0.437644,0.996137,0.135570,0.981189,0.341188,-0.958854
oper_valor_carencia,0.171839,-0.045006,0.066276,-0.102671,-0.879206,0.177699,0.003584,0.133850,1.000000,-0.074752,0.117701,0.952575,0.120064,0.778515,0.153003
oper_spreadbndes_juros,0.087102,-0.340261,-0.994218,0.456919,0.048665,0.175508,0.926670,-0.437644,-0.074752,1.000000,-0.451277,-0.129586,-0.588166,-0.639969,0.415003


<a name="partition"></a>
    
## 1.4) Train/Test partition 

- ir para [Menu Principal](#navegacao)

In [74]:
%%time
files = spread.columns
selected_files = files.str.contains('firma|oper|banco')
features = files[selected_files]
lista = list(features)
#lista.remove('prazo_carencia')
lista.sort()

Wall time: 0 ns


In [75]:
lista

['banco_ativo',
 'banco_ativo_captacoes',
 'banco_ativo_carteira',
 'banco_ativo_passivo',
 'banco_ativo_patrimonio',
 'banco_b1',
 'banco_b2',
 'banco_b3',
 'banco_captacoes',
 'banco_carteira',
 'banco_carteira_passivo',
 'banco_enc',
 'banco_lucroliquido',
 'banco_numagencias',
 'banco_numeropostos',
 'banco_passivo',
 'banco_patrimonio',
 'firma_enc',
 'firma_engen',
 'firma_idade',
 'firma_idade_engen',
 'firma_idade_massa',
 'firma_idade_po3gr',
 'firma_massa',
 'firma_massa_po3gr',
 'firma_po3gr',
 'firma_potot',
 'firma_potot_idade',
 'firma_potot_massa',
 'firma_potot_po3gr',
 'firma_rotat',
 'firma_share',
 'oper_amortizacao',
 'oper_amortizacao_carencia',
 'oper_carencia',
 'oper_juros',
 'oper_juros_amortizacao',
 'oper_juros_carencia',
 'oper_spreadbndes',
 'oper_spreadbndes_amortizacao',
 'oper_spreadbndes_carencia',
 'oper_spreadbndes_juros',
 'oper_valor',
 'oper_valor_amortizacao',
 'oper_valor_carencia',
 'oper_valor_juros',
 'oper_valor_spreadbndes']

In [76]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingRandomSearchCV, HalvingGridSearchCV
import xgboost as xgb 

X, y = spread[lista] , spread[['target']]

Xmin_max = MinMaxScaler().fit_transform(X)
ymin_max = MinMaxScaler().fit_transform(y)

In [77]:
import sklearn
print(sklearn.__version__)

0.24.2


In [78]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

X_train, X_test, y_train, y_test = train_test_split(X , y , test_size=0.30)

X_train0, X_test0, y_train0, y_test0 = train_test_split(Xmin_max , ymin_max , test_size=0.30)

<a name="randomsearch"></a>

# 2) Random Search for Hyperparameters

- 2.1 [Extreme Learning Machine](#elm1)
- 2.2 [Suport Vector Machine](#svm1)
- 2.3 [Stochastic Gradient Descent](#sgd1)
- 2.4 [Multi-Task Elastic-Net](#elastic1)
- 2.5 [Least Angle Regression Model (LAR)](#LAR1)
- 2.6 [Ridge Regression](#ridge1)
- 2.7 [ElasticNet Regression](#ElasNet1)
- 2.8 [Bayesian Ridge](#bayesian1)
- 2.9 [Lasso Regression](#lasso1)
- 2.10 [Multilayer Perceptron Regression](#mlp1)
- 2.11 [Nearest Neighbors Regression](#nnr1)
- 2.12 [Resumo dos resultados](#resumo_rs1)

<a name="svm1"></a>

## 2.1) Extreme Gradient Boosting (xgboost)

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [34]:
import xgboost as xgb
space = dict()
space['booster']           = ['gbtree','gblinear','dart']
space['colsample_bylevel'] = [row/50 for row in range(1,100,10)]
#space['colsample_bynode']  = [row/50 for row in range(1,100,10)]
#space['colsample_bytree']  = [row/50 for row in range(1,100,10)]
space['gamma']             = [row/30 for row in range(1,30,2)]
space['importance_type']   = ['gain']
space['learning_rate']     = [row/400 for row in range(1,200,10)]
space['max_delta_step']    = [row/50 for row in range(1,100,5)]
space['max_depth']         = [row for row in range(1,10,2)]
space['min_child_weight']  = [row/1000 for row in range(1,100,20)]
space['reg_lambda']        = [row/1000 for row in range(1,100,20)]
space['reg_alpha']         = [row/1000 for row in range(1,100,20)]
space['subsample']         = [row/200  for row in range(1,100,20)]
space['objective']         = ['reg:linear']

xgboost = xgb.XGBRegressor(n_estimators=100,verbosity=0)
xgboost1= RandomizedSearchCV(xgboost, space ,  cv=3, random_state=1, n_iter=300, n_jobs=20)
xgboost1.fit(X_train, y_train)

RandomizedSearchCV(cv=3,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=100, n...
                                        'max_delta_step': [0.02, 0.12, 0.22,
                                                           0.32, 0.42, 0.52,
                                    

In [35]:
# summarize result
print('Best Score: %s' % xgboost1.best_score_) #Best Score: 0.9352753839517703
print('Best Hyperparameters: %s' % xgboost1.best_params_)

Best Score: 0.9956248568887562
Best Hyperparameters: {'subsample': 0.205, 'reg_lambda': 0.041, 'reg_alpha': 0.001, 'objective': 'reg:linear', 'min_child_weight': 0.021, 'max_depth': 9, 'max_delta_step': 1.42, 'learning_rate': 0.1275, 'importance_type': 'gain', 'gamma': 0.36666666666666664, 'colsample_bylevel': 0.82, 'booster': 'gbtree'}


In [36]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error
print("R2:", r2_score(                 y_test ,xgboost1.predict(X_test)))
print("EVS", explained_variance_score( y_test ,xgboost1.predict(X_test)))
print("MAE", mean_absolute_error(      y_test ,xgboost1.predict(X_test)))

R2: 0.9958264871075617
EVS 0.995826644923932
MAE 0.11971137321207123


In [55]:
xgboost1_ = xgb.XGBRegressor(n_estimators=100, verbosity=0, subsample=0.205, reg_lambda=0.041, reg_alpha=0.001,
                             objective='reg:linear', min_child_weight=0.021, max_depth=9, max_delta_step=1.42,
                             learning_rate=0.1275, importance_type='gain', gamma=0.36666666666666664, 
                             colsample_bylevel=0.82, booster='gbtree')

xgboost1_.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.82,
             colsample_bynode=1, colsample_bytree=1, gamma=0.36666666666666664,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.1275, max_delta_step=1.42, max_depth=9,
             min_child_weight=0.021, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=0.001,
             reg_lambda=0.041, scale_pos_weight=1, subsample=0.205,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [56]:
dataframe=pd.DataFrame(xgboost1_.feature_importances_, columns=['feature_importances']) 
dataframe['features'] = lista
dataframe.sort_values('feature_importances')

,feature_importances,features
7,0.000000,banco_b3
15,0.000000,banco_passivo
46,0.000500,oper_valor_spreadbndes
5,0.000506,banco_b1
36,0.000543,oper_juros_amortizacao
17,0.000580,firma_enc
18,0.000585,firma_engen
12,0.000588,banco_lucroliquido
43,0.000603,oper_valor_amortizacao
16,0.000614,banco_patrimonio


In [57]:
import pickle
with open("pickle/xgboost1_.pkl", 'wb') as file:
    pickle.dump(xgboost1_, file)

<a name="sgd1"></a>

## 3.2) Stochastic Gradient Descent 

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [31]:
%%time
from sklearn.linear_model import SGDRegressor

# define search space
space = dict()
space['alpha']          = [row/50 for row in range(40,500,25)]
space['l1_ratio']       = [row/400 for row in range(20,100,4)]
space['loss']           = ['squared_loss','huber','epsilon_insensitive','squared_epsilon_insensitive']
space['penalty']        = ['l1','l2','elasticnet']
space['learning_rate']  = ['invscaling','constant','optimal','adaptive']
space['epsilon']        = [row/700 for row in range(10,200,20)]
space['eta0']           = [row/1200 for row in range(1,200,10)]
space['power_t']        = [row/600 for row in range(60,300,10)]
space['early_stopping'] = [False,True]
space['tol']            = [row/5000 for row in range(1,60,5)]

sgd = SGDRegressor(max_iter=1000)

sgd1 = RandomizedSearchCV(sgd, space, cv=3, random_state=666, n_iter=300, n_jobs=20)

sgd1.fit(X_train0, y_train0.ravel())

Wall time: 1min 55s


RandomizedSearchCV(cv=3, estimator=SGDRegressor(), n_iter=300, n_jobs=20,
                   param_distributions={'alpha': [0.8, 1.3, 1.8, 2.3, 2.8, 3.3,
                                                  3.8, 4.3, 4.8, 5.3, 5.8, 6.3,
                                                  6.8, 7.3, 7.8, 8.3, 8.8, 9.3,
                                                  9.8],
                                        'early_stopping': [False, True],
                                        'epsilon': [0.014285714285714285,
                                                    0.04285714285714286,
                                                    0.07142857142857142, 0.1,
                                                    0.12857142857142856,
                                                    0.15714285714285714,
                                                    0.18571428571428572,
                                                    0.21428...
                                                  

In [40]:
# summarize result
print('Best Score: %s' % sgd1.best_score_) 
print('Best Hyperparameters: %s' % sgd1.best_params_)

Best Score: 0.33285258485516095
Best Hyperparameters: {'tol': 0.0062, 'power_t': 0.48333333333333334, 'penalty': 'l2', 'loss': 'huber', 'learning_rate': 'adaptive', 'l1_ratio': 0.09, 'eta0': 0.08416666666666667, 'epsilon': 0.2714285714285714, 'early_stopping': False, 'alpha': 0.8}


In [41]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error
print("R2:", r2_score(                 y_test0.ravel(),sgd1.predict(X_test0)))
print("EVS", explained_variance_score( y_test0.ravel(),sgd1.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0.ravel(),sgd1.predict(X_test0)))

R2: 0.3301238389121065
EVS 0.3301257348591087
MAE 0.09829092080014325


In [42]:
sgd1_ = SGDRegressor(max_iter=800, warm_start=True, tol=0.0062, power_t= 0.48333333333333334, penalty= 'l2',
                    loss='huber', learning_rate= 'adaptive', l1_ratio= 0.09, eta0=0.08416666666666667,
                    epsilon= 0.2714285714285714, early_stopping=False, alpha=0.8)

sgd1_.fit(X_train, y_train)

SGDRegressor(alpha=0.8, epsilon=0.2714285714285714, eta0=0.08416666666666667,
             l1_ratio=0.09, learning_rate='adaptive', loss='huber',
             max_iter=800, power_t=0.48333333333333334, tol=0.0062,
             warm_start=True)

In [43]:
dataframe=pd.DataFrame(sgd1_.coef_, columns=['feature_importances']) 
dataframe['features'] = lista
dataframe.sort_values('feature_importances')

,feature_importances,features
33,-0.013711,oper_amortizacao_carencia
45,-0.007753,oper_valor_juros
23,-0.004873,firma_massa
41,-0.004664,oper_spreadbndes_juros
42,-0.003236,oper_valor
46,-0.003089,oper_valor_spreadbndes
26,-0.003075,firma_potot
25,-0.002581,firma_po3gr
44,-0.002060,oper_valor_carencia
0,-0.001152,banco_ativo


In [44]:
#dataframe=pd.DataFrame(sgd1_.feature_importances_, columns=['feature_importances']) 
#dataframe['features'] = lista
#dataframe.sort_values('feature_importances')

In [45]:
import pickle
with open("pickle/sgd1_.pkl", 'wb') as file:
    pickle.dump(sgd1_, file)

<a name="elastic1"></a>

## 2.3) Multi-Task Elastic-Net

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [7]:
print("mínimo",min([row/5000 for row in range(1,500,10)]),"/n","máximo",max([row/5000 for row in range(1,500,10)]))

mínimo 0.0002 /n máximo 0.0982


by 0.002


In [46]:
%%time
from sklearn import linear_model
space = dict()
space['alpha']         = [row/200 for row in range(1,1000,50)]
space['l1_ratio']      = [row/100 for row in range(1,200,10)]
space['selection']     = ['cyclic','random']
space['tol']           = [row/5000 for row in range(1,500,10)]

mtask = linear_model.MultiTaskElasticNet(max_iter=800)

mtask1 = RandomizedSearchCV(mtask, space, cv=3,  n_iter=300, random_state=1, n_jobs=20)

mtask1.fit(X_train, y_train)

Wall time: 1min 3s


RandomizedSearchCV(cv=3, estimator=MultiTaskElasticNet(max_iter=800),
                   n_iter=300, n_jobs=20,
                   param_distributions={'alpha': [0.005, 0.255, 0.505, 0.755,
                                                  1.005, 1.255, 1.505, 1.755,
                                                  2.005, 2.255, 2.505, 2.755,
                                                  3.005, 3.255, 3.505, 3.755,
                                                  4.005, 4.255, 4.505, 4.755],
                                        'l1_ratio': [0.01, 0.11, 0.21, 0.31,
                                                     0.41, 0.51, 0.61, 0.71,
                                                     0.81, 0.91, 1.01, 1.11,
                                                     1.21, 1.31, 1.41, 1.51,
                                                     1.61, 1.71, 1.81, 1.91],
                                        'selection': ['cyclic', 'random'],
                                    

In [47]:
# summarize result
print('Best Score: %s' % mtask1.best_score_)
print('Best Hyperparameters: %s' % mtask1.best_params_)

Best Score: 0.9912287539349389
Best Hyperparameters: {'tol': 0.0302, 'selection': 'cyclic', 'l1_ratio': 0.01, 'alpha': 0.005}


In [48]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test,mtask1.predict(X_test)))
print("EVS", explained_variance_score( y_test,mtask1.predict(X_test)))
print("MAE", mean_absolute_error(      y_test,mtask1.predict(X_test)))

R2: 0.9913230439081949
EVS 0.9913231564644762
MAE 0.2212489235520475


In [49]:
mtask1_ = linear_model.MultiTaskElasticNet(max_iter=800, warm_start=True, tol=0.0302, selection='cyclic',
                                           l1_ratio=0.01, alpha=0.005)

mtask1_.fit(X_train, y_train)

MultiTaskElasticNet(alpha=0.005, l1_ratio=0.01, max_iter=800, tol=0.0302,
                    warm_start=True)

In [50]:
dataframe=pd.DataFrame(mtask1_.coef_[0], columns=['feature_importances']) 
dataframe['features'] = lista
dataframe.sort_values('feature_importances')

,feature_importances,features
38,-5.091624e-01,oper_spreadbndes
32,-1.585182e-01,oper_amortizacao
41,-1.431886e-01,oper_spreadbndes_juros
34,-1.105808e-01,oper_carencia
10,-1.054244e-01,banco_carteira_passivo
6,-8.435480e-02,banco_b2
33,-7.785609e-02,oper_amortizacao_carencia
15,-5.614511e-02,banco_passivo
36,-5.191999e-02,oper_juros_amortizacao
16,-3.375367e-02,banco_patrimonio


In [51]:
import pickle
with open("pickle/mtask1_.pkl", 'wb') as file:
    pickle.dump(mtask1_, file)

In [52]:
#dataframe=pd.DataFrame(mtask00.feature_importances_, columns=['feature_importances']) 
#dataframe['features'] = lista
#dataframe.sort_values('feature_importances')

<a name="LAR1"></a>

## 2.4) Least Angle Regression Model (LAR)

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [58]:
%%time
from sklearn import linear_model
space = dict()
space['alpha']          = [row/300 for row in range(1,900,10)]
space['fit_intercept']  = [False,True]
space['fit_path']       = [False,True]
space['normalize']      = [False,True]
space['copy_X']         = [False,True]
space['positive']       = [False,True]
space['eps']            = [row/100 for row in range(1000,10000,100)]

lars = linear_model.LassoLars()

lars1 = RandomizedSearchCV(lars, space, cv=3,  n_iter=300, random_state=666, n_jobs=20)

lars1.fit(X_train, y_train)

Wall time: 49.8 s


RandomizedSearchCV(cv=3, estimator=LassoLars(), n_iter=300, n_jobs=20,
                   param_distributions={'alpha': [0.0033333333333333335,
                                                  0.03666666666666667, 0.07,
                                                  0.10333333333333333,
                                                  0.13666666666666666, 0.17,
                                                  0.20333333333333334,
                                                  0.23666666666666666, 0.27,
                                                  0.30333333333333334,
                                                  0.33666666666666667, 0.37,
                                                  0.4033333333333333,
                                                  0.43666666666666665, 0.47,
                                                  0.5033333333333333,
                                                  0.536666...
                                                  0.9033

In [59]:
# summarize result
print('Best Score: %s' % lars1.best_score_) #Best Score: 0.9471256547797681
print('Best Hyperparameters: %s' % lars1.best_params_)

Best Score: 0.9908355150966112
Best Hyperparameters: {'positive': False, 'normalize': True, 'fit_path': False, 'fit_intercept': False, 'eps': 22.0, 'copy_X': False, 'alpha': 0.03666666666666667}


In [60]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error
print("R2:", r2_score(                 y_test,lars1.predict(X_test)))
print("EVS", explained_variance_score( y_test,lars1.predict(X_test)))
print("MAE", mean_absolute_error(      y_test,lars1.predict(X_test)))

R2: 0.9909200361872162
EVS 0.9909200525814799
MAE 0.22687695045666006


In [61]:
lars1_ = linear_model.LassoLars(max_iter=800, positive=False, normalize=True, fit_path=False,
                                fit_intercept= False, eps= 22.0, copy_X= False, alpha= 0.0366667)
    
lars1_.fit(X_train, y_train)

LassoLars(alpha=0.0366667, copy_X=False, eps=22.0, fit_intercept=False,
          fit_path=False, max_iter=800)

In [62]:
dataframe=pd.DataFrame(lars1_.coef_[0], columns=['feature_importances']) 
dataframe['features'] = lista
dataframe.sort_values('feature_importances')

,feature_importances,features
41,-7.760257e-01,oper_spreadbndes_juros
45,-1.883718e-02,oper_valor_juros
24,-1.695319e-03,firma_massa_po3gr
17,-5.244911e-07,firma_enc
25,0.000000e+00,firma_po3gr
26,0.000000e+00,firma_potot
27,0.000000e+00,firma_potot_idade
28,0.000000e+00,firma_potot_massa
29,0.000000e+00,firma_potot_po3gr
30,0.000000e+00,firma_rotat


In [63]:
import pickle
with open("pickle/lars1_.pkl", 'wb') as file:
    pickle.dump(lars1_, file)

<a name="ridge1"></a>

## 2.5) Ridge Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [64]:
%%time
from sklearn.linear_model import Ridge
  
space = dict()
space['alpha']         = [row/500 for row in range(8,100,5)]
space['copy_X']        = [False,True]
space['fit_intercept'] = [False,True]
space['normalize']     = [False,True]
space['solver']        = ['auto','svd','cholesky','lsqr','sparse_cg']
space['tol']           = [row/100000 for row in range(1,1000,10)]

ridge = Ridge()

ridge1 = RandomizedSearchCV(ridge, space, cv=3,  n_iter=30 , random_state=666, n_jobs=20)

ridge1.fit(X_train, y_train)

Wall time: 7.25 s


RandomizedSearchCV(cv=3, estimator=Ridge(), n_iter=30, n_jobs=20,
                   param_distributions={'alpha': [0.016, 0.026, 0.036, 0.046,
                                                  0.056, 0.066, 0.076, 0.086,
                                                  0.096, 0.106, 0.116, 0.126,
                                                  0.136, 0.146, 0.156, 0.166,
                                                  0.176, 0.186, 0.196],
                                        'copy_X': [False, True],
                                        'fit_intercept': [False, True],
                                        'normalize': [False, True],
                                        'solver': ['auto', 'svd', 'cholesky',
                                                   'lsqr', 'sparse_cg'],
                                        'tol': [1e-05, 0.00011, 0.00021,
                                                0.00031, 0.00041, 0.00051,
                                              

In [65]:
# summarize result
print('Best Score: %s' % ridge1.best_score_) #Best Score: 0.9917155408859822
print('Best Hyperparameters: %s' % ridge1.best_params_)

Best Score: 0.9917322520636879
Best Hyperparameters: {'tol': 0.00401, 'solver': 'svd', 'normalize': True, 'fit_intercept': False, 'copy_X': True, 'alpha': 0.036}


In [66]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error
print("R2:", r2_score(                 y_test, ridge1.predict(X_test)))
print("EVS", explained_variance_score( y_test, ridge1.predict(X_test)))
print("MAE", mean_absolute_error(      y_test, ridge1.predict(X_test)))

R2: 0.9917390603353198
EVS 0.9917392322429371
MAE 0.21642389215178182


In [67]:
ridge1_ = Ridge(tol=0.00401, solver='svd', normalize=True, fit_intercept=False, copy_X=True, alpha=0.036)
    
ridge1_.fit(X_train, y_train)

Ridge(alpha=0.036, fit_intercept=False, normalize=True, solver='svd',
      tol=0.00401)

In [68]:
dataframe=pd.DataFrame(ridge1_.coef_[0], columns=['feature_importances']) 
dataframe['features'] = lista
dataframe.sort_values('feature_importances')

,feature_importances,features
10,-3.796494e+00,banco_carteira_passivo
1,-3.781824e+00,banco_ativo_captacoes
31,-1.174998e+00,firma_share
8,-5.572701e-01,banco_captacoes
15,-3.650256e-01,banco_passivo
3,-3.123441e-01,banco_ativo_passivo
41,-2.935645e-01,oper_spreadbndes_juros
45,-1.604179e-01,oper_valor_juros
40,-1.418083e-01,oper_spreadbndes_carencia
39,-1.418030e-01,oper_spreadbndes_amortizacao


In [69]:
import pickle
# Save to file in the current working directory
with open("pickle/ridge1_.pkl", 'wb') as file:
    pickle.dump(ridge1_, file)

<a name="ElasNet1"></a>

## 2.6) ElasticNet Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [30]:
lista = [row/10000 for row in range(10,10000,10)]
print("mínimo",min(lista),"/n","máximo",max(lista))

mínimo 0.001 /n máximo 0.999


In [31]:
print("by", 10/10000)

by 0.001


In [70]:
%%time
from sklearn.linear_model import ElasticNet
space = dict()
space['alpha']         = [row/500 for row in range(1,500,5)]
space['l1_ratio']      = [row/250 for row in range(5,1000,10)]
space['fit_intercept'] = [False,True]
space['normalize']     = [False,True]
space['precompute']    = [False,True]
space['copy_X']        = [False,True]
space['warm_start']    = [False,True]
space['positive']      = [False,True]
space['selection']     = ['cyclic','random']
space['tol']           = [row/10000 for row in range(10,10000,10)]

elastic = ElasticNet()

elastic1 = RandomizedSearchCV(elastic, space, cv=3,  n_iter=300, random_state=666, n_jobs=20)

elastic1.fit(X_train0, y_train0)

Wall time: 15.3 s


RandomizedSearchCV(cv=3, estimator=ElasticNet(), n_iter=300, n_jobs=20,
                   param_distributions={'alpha': [0.002, 0.012, 0.022, 0.032,
                                                  0.042, 0.052, 0.062, 0.072,
                                                  0.082, 0.092, 0.102, 0.112,
                                                  0.122, 0.132, 0.142, 0.152,
                                                  0.162, 0.172, 0.182, 0.192,
                                                  0.202, 0.212, 0.222, 0.232,
                                                  0.242, 0.252, 0.262, 0.272,
                                                  0.282, 0.292, ...],
                                        'copy_X': [False, True],
                                        'fit_intercept': [False, True],
                                        'l1_ratio': [0.02...
                                                     1.14, 1.18, ...],
                                        'no

In [71]:
# summarize result
print('Best Score: %s' % elastic1.best_score_) #Best Score: 0.6300884516001556
print('Best Hyperparameters: %s' % elastic1.best_params_)

Best Score: 0.661742049052565
Best Hyperparameters: {'warm_start': False, 'tol': 0.451, 'selection': 'cyclic', 'precompute': True, 'positive': False, 'normalize': False, 'l1_ratio': 0.02, 'fit_intercept': False, 'copy_X': True, 'alpha': 0.022}


In [72]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error
print("R2:", r2_score(                 y_test0, elastic1.predict(X_test0)))
print("EVS", explained_variance_score( y_test0, elastic1.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0, elastic1.predict(X_test0)))

R2: 0.6555790202450884
EVS 0.6567235208982091
MAE 0.07452988622890028


In [73]:
elastic1_ = ElasticNet(max_iter=800, warm_start=False, tol=0.451, selection='cyclic',
                       normalize=False, precompute= True, positive=False,l1_ratio=0.02,
                       fit_intercept=False, copy_X=False, alpha=0.022)

elastic1_.fit(X_train, y_train)

ElasticNet(alpha=0.022, copy_X=False, fit_intercept=False, l1_ratio=0.02,
           max_iter=800, precompute=True, tol=0.451)

In [74]:
import pickle
# Save to file in the current working directory
with open("pickle/elastic1_.pkl", 'wb') as file:
    pickle.dump(elastic1_, file)

<a name="bayesian1"></a>

## 3.7) Bayesian Ridge Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [ ]:
print("by", 10/10000)

In [75]:
%%time
from sklearn.linear_model import BayesianRidge

space = dict()
space['alpha_1']        = [row/40000000 for row in range(1,2000,50)]
space['alpha_2']        = [row/40000000 for row in range(1,2000,50)]
space['lambda_1']       = [row/40000000 for row in range(1,2000,50)]
space['lambda_2']       = [row/40000000 for row in range(1,2000,50)]
space['compute_score']  = [False,True]
space['copy_X']         = [False,True]
space['fit_intercept']  = [False,True]
space['normalize']      = [False,True]
space['tol']            = [row/5000 for row in range(1,100,5)]

bayes = BayesianRidge(n_iter=800)

bayes1 = RandomizedSearchCV(bayes, space, cv=3,  n_iter=300, random_state=666, n_jobs=20)

bayes1.fit(X_train0, y_train0.ravel())

Wall time: 2min 38s


RandomizedSearchCV(cv=3, estimator=BayesianRidge(n_iter=800), n_iter=300,
                   n_jobs=20,
                   param_distributions={'alpha_1': [2.5e-08, 1.275e-06,
                                                    2.525e-06, 3.775e-06,
                                                    5.025e-06, 6.275e-06,
                                                    7.525e-06, 8.775e-06,
                                                    1.0025e-05, 1.1275e-05,
                                                    1.2525e-05, 1.3775e-05,
                                                    1.5025e-05, 1.6275e-05,
                                                    1.7525e-05, 1.8775e-05,
                                                    2.0025e-05, 2.1275e-05,
                                                    2.2525e-05, 2.3775e-05,
                                                    2.5025e-05, 2.6275e-05,
                                                    2.752...
         

In [76]:
# summarize result
print('Best Score: %s' % bayes1.best_score_)
print('Best Hyperparameters: %s' % bayes1.best_params_)

Best Score: 0.9917775045815752
Best Hyperparameters: {'tol': 0.0002, 'normalize': True, 'lambda_2': 4.8775e-05, 'lambda_1': 3.775e-06, 'fit_intercept': False, 'copy_X': True, 'compute_score': False, 'alpha_2': 1.0025e-05, 'alpha_1': 4.6275e-05}


In [77]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error
print("R2:", r2_score(                 y_test0, bayes1.predict(X_test0)))
print("EVS", explained_variance_score( y_test0, bayes1.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0, bayes1.predict(X_test0)))

R2: 0.9916629054491023
EVS 0.9916650195397624
MAE 0.01183298047677855


In [78]:
bayes1_ = BayesianRidge(tol=0.0002, normalize=True, lambda_2=4.8775e-05, lambda_1=3.775e-06, fit_intercept=False,
                        copy_X=False, compute_score= False, alpha_2= 1.0025e-05, alpha_1= 4.6275e-05)
 
bayes1_.fit(X_train, y_train)

BayesianRidge(alpha_1=4.6275e-05, alpha_2=1.0025e-05, copy_X=False,
              fit_intercept=False, lambda_1=3.775e-06, lambda_2=4.8775e-05,
              normalize=True, tol=0.0002)

In [79]:
import pickle
# Save to file in the current working directory
with open("pickle/bayes1_.pkl", 'wb') as file:
    pickle.dump(bayes1_, file)

<a name="lasso1"></a>

## 3.8) Lasso Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [35]:
lista = [row/1000000 for row in range(100,10000,10)]
print("mínimo",min(lista),"/n","máximo",max(lista))

mínimo 0.0001 /n máximo 0.00999


In [36]:
print("by",10/1000000)

by 1e-05


In [80]:
%%time
from sklearn.linear_model import Lasso
space = dict()
space['alpha']         = [row/1000 for row in range(10,100,5)]
space['copy_X']        = [False,True]
space['fit_intercept'] = [False,True]
space['normalize']     = [False,True]
space['precompute']    = [False,True]
space['positive']      = [False,True]
space['selection']     = ['cyclic','random']
space['tol']           = [row/1000000 for row in range(100,10000,10)]

lasso = Lasso()

lasso1 = RandomizedSearchCV(lasso, space, cv=3, n_iter=300, random_state=666, n_jobs=20)

lasso1.fit(X_train0, y_train0)

Wall time: 39.8 s


RandomizedSearchCV(cv=3, estimator=Lasso(), n_iter=300, n_jobs=20,
                   param_distributions={'alpha': [0.01, 0.015, 0.02, 0.025,
                                                  0.03, 0.035, 0.04, 0.045,
                                                  0.05, 0.055, 0.06, 0.065,
                                                  0.07, 0.075, 0.08, 0.085,
                                                  0.09, 0.095],
                                        'copy_X': [False, True],
                                        'fit_intercept': [False, True],
                                        'normalize': [False, True],
                                        'positive': [False, True],
                                        'precompute': [False, True],
                                        'selection': ['cyclic', 'random'],
                                        'tol': [0.0001, 0.00011, 0.00012,
                                                0.00013, 0.00014, 0.00015,


In [81]:
# summarize result
print('Best Score: %s' % lasso1.best_score_) #Best Score: 0.8776039138486295
print('Best Hyperparameters: %s' % lasso1.best_params_) 

Best Score: 0.9036186347365248
Best Hyperparameters: {'tol': 0.00233, 'selection': 'cyclic', 'precompute': True, 'positive': False, 'normalize': False, 'fit_intercept': False, 'copy_X': True, 'alpha': 0.01}


In [82]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error
print("R2:", r2_score(                 y_test0, lasso1.predict(X_test0)))
print("EVS", explained_variance_score( y_test0, lasso1.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0, lasso1.predict(X_test0)))

R2: 0.9035177053904724
EVS 0.9075576689954088
MAE 0.03621968003647397


In [83]:
lasso1_ = Lasso(max_iter=800, warm_start=True, tol=0.00233, selection='cyclic', precompute=True,
                normalize=False, fit_intercept=False, copy_X=False, alpha=0.01)

                                
lasso1_.fit(X_train, y_train)

Lasso(alpha=0.01, copy_X=False, fit_intercept=False, max_iter=800,
      precompute=True, tol=0.00233, warm_start=True)

In [84]:
import pickle
# Save to file in the current working directory
with open("pickle/lasso1_.pkl", 'wb') as file:
    pickle.dump(lasso1_, file)

<a name="mlp1"></a>

## 3.9) Multilayer Perceptron

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [49]:
lista = [row/200 for row in range(1,20,6)]
print("mínimo",min(lista),"/n","máximo",max(lista))

mínimo 0.005 /n máximo 0.095


In [50]:
print("by",6/200)

by 0.03


In [31]:
%%time
from sklearn.neural_network import MLPRegressor
space = dict()
space['hidden_layer_sizes']  = [(100,50), (200,25), (25,100,25), (50,50,50), (75,50,75), (80,60,80)]
space['activation']          = ['identity','logistic','tanh','relu']
space['solver']              = ['lbfgs', 'sgd', 'adam']
space['alpha']               = [row/200 for row in range(1,100,10)]
space['learning_rate']       = ['constant','invscaling','adaptive']
space['learning_rate_init']  = [row/1000 for row in range(1,20,8)]
space['warm_start']          = [False,True]
space['power_t']             = [row/50 for row in range(1,200,20)]
space['tol']                 = [row/50000000 for row in range(1,1000,200)]
space['momentum']            = [row/200 for row in range(1,100,10)]
space['nesterovs_momentum']  = [False,True]
space['early_stopping']      = [False,True]
space['beta_1']              = [row/200 for row in range(1,20,6)]
space['beta_2']              = [row/200 for row in range(1,20,6)]
space['epsilon']             = [row/200 for row in range(1,20,6)]
    
mpl = MLPRegressor(max_iter=1000)

mpl1 = RandomizedSearchCV(mpl, space, cv=3,  n_iter=100, random_state=666, n_jobs=10)

mpl1.fit(X_train0, y_train0.ravel())

Wall time: 6h 39min 14s


RandomizedSearchCV(cv=3, estimator=MLPRegressor(max_iter=1000), n_iter=100,
                   n_jobs=10,
                   param_distributions={'activation': ['identity', 'logistic',
                                                       'tanh', 'relu'],
                                        'alpha': [0.005, 0.055, 0.105, 0.155,
                                                  0.205, 0.255, 0.305, 0.355,
                                                  0.405, 0.455],
                                        'beta_1': [0.005, 0.035, 0.065, 0.095],
                                        'beta_2': [0.005, 0.035, 0.065, 0.095],
                                        'early_stopping': [False, True],
                                        'epsilon': [0.005, 0.035, 0....
                                                          'adaptive'],
                                        'learning_rate_init': [0.001, 0.009,
                                                               0.017]

In [32]:
# summarize result
print('Best Score: %s' % mpl1.best_score_) #Best Score: 0.9550165534562787
print('Best Hyperparameters: %s' % mpl1.best_params_)

Best Score: 0.9932967582667148
Best Hyperparameters: {'warm_start': True, 'tol': 8.02e-06, 'solver': 'lbfgs', 'power_t': 2.02, 'nesterovs_momentum': True, 'momentum': 0.055, 'learning_rate_init': 0.001, 'learning_rate': 'constant', 'hidden_layer_sizes': (200, 25), 'epsilon': 0.035, 'early_stopping': False, 'beta_2': 0.065, 'beta_1': 0.005, 'alpha': 0.005, 'activation': 'relu'}


In [34]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test0, mpl1.predict(X_test0)))
print("EVS", explained_variance_score( y_test0, mpl1.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0, mpl1.predict(X_test0)))

R2: 0.9935653377571071
EVS 0.9935653381771745
MAE 0.009327471920636017


In [35]:
mpl1_ = MLPRegressor(max_iter=800, warm_start=True, tol=8.02e-06, solver='lbfgs', power_t=2.02,
                       nesterovs_momentum=True, momentum=0.055, learning_rate_init=0.001, 
                       learning_rate='constant', hidden_layer_sizes=(200, 25), epsilon=0.035, 
                       early_stopping=False, beta_2=0.065, beta_1=0.005, alpha=0.005, activation='relu')

mpl1_.fit(X_train, y_train)

MLPRegressor(alpha=0.005, beta_1=0.005, beta_2=0.065, epsilon=0.035,
             hidden_layer_sizes=(200, 25), max_iter=800, momentum=0.055,
             power_t=2.02, solver='lbfgs', tol=8.02e-06, warm_start=True)

In [36]:
import pickle
# Save to file in the current working directory
with open("pickle/mpl1_.pkl", 'wb') as file:
    pickle.dump(mpl1_, file)

<a name="nnr1"></a>

## 3.10) Nearest Neighbors Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [52]:
lista = [row for row in range(5,35)]
print("mínimo",min(lista),"/n","máximo",max(lista))

mínimo 5 /n máximo 34


In [ ]:
print("by",6/350)

In [ ]:
%%time
from sklearn.neighbors import KNeighborsRegressor

space = dict()
space['n_neighbors'] = [row for row in range(4,10)]
space['weights']     = ['uniform', 'distance']
space['algorithm']   = ['auto', 'ball_tree', 'kd_tree', 'brute']
space['leaf_size']   = [row for row in range(5,35)]
space['p']           = [row for row in range(2,8)]

knn = KNeighborsRegressor()

knn1 = RandomizedSearchCV(knn, space, cv=3, random_state=666, n_iter=100, n_jobs=10)

knn1.fit(X_train0, y_train0)

In [ ]:
# summarize result
print('Best Score: %s' % knn1.best_score_)
print('Best Hyperparameters: %s' % knn1.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error
print("R2:", r2_score(                 y_test0, knn1.predict(X_test0)))
print("EVS", explained_variance_score( y_test0, knn1.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0, knn1.predict(X_test0)))

In [ ]:
knn1_ = KNeighborsRegressor(max_iter=800, warm_start=True, tol=0.0006, selection='cyclic',normalize=False,
                                           l1_ratio=1.0333333333333334, fit_intercept=False, copy_X=False, alpha=0.005)


knn1_.fit(X_train, y_train)

In [ ]:
import pickle
# Save to file in the current working directory
with open("pickle/knn1_.pkl", 'wb') as file:
    pickle.dump(knn1_, file)

<a name="resumo_rs1"></a>

## 2.11) Resumo dos resultados

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [83]:
import pickle
xgboost   = pickle.load(open('pickle/xgboost1_.pkl', 'rb'))
sgd       = pickle.load(open('pickle/sgd1_.pkl', 'rb'))
mtask     = pickle.load(open('pickle/mtask1_.pkl', 'rb'))
lars      = pickle.load(open('pickle/lars1_.pkl', 'rb'))
ridge     = pickle.load(open('pickle/ridge1_.pkl', 'rb'))    
elastic   = pickle.load(open('pickle/elastic1_.pkl', 'rb'))
bayes     = pickle.load(open('pickle/bayes1_.pkl', 'rb'))
lasso     = pickle.load(open('pickle/lasso1_.pkl', 'rb'))
mpl       = pickle.load(open('pickle/mpl1_.pkl', 'rb'))
#knn       = pickle.load(open('pickle/knn1_.pkl', 'rb'))

In [89]:
R2_xgboost  = r2_score(                 y_test ,xgboost.predict(X_test))
EVS_xgboost = explained_variance_score( y_test ,xgboost.predict(X_test))
MAE_xgboost = mean_absolute_error(      y_test ,xgboost.predict(X_test))

#R2_sgd      = r2_score(                 y_test0 ,sgd1_.predict(X_test0))
#EVS_sgd     = explained_variance_score( y_test0 ,sgd1_.predict(X_test0))
#MAE_sgd     = mean_absolute_error(      y_test0 ,sgd1_.predict(X_test0))
#
#R2_mtask    = r2_score(                 y_test  ,mtask1_.predict(X_test))
#EVS_mtask   = explained_variance_score( y_test  ,mtask1_.predict(X_test))
#MAE_mtask   = mean_absolute_error(      y_test  ,mtask1_.predict(X_test))
#
#R2_lars     = r2_score(                 y_test  ,lars1_.predict(X_test))
#EVS_lars    = explained_variance_score( y_test  ,lars1_.predict(X_test))
#MAE_lars    = mean_absolute_error(      y_test  ,lars1_.predict(X_test))
#
#R2_ridge    = r2_score(                 y_test  ,ridge1_.predict(X_test))
#EVS_ridge   = explained_variance_score( y_test  ,ridge1_.predict(X_test))
#MAE_ridge   = mean_absolute_error(      y_test  ,ridge1_.predict(X_test))
#
#R2_elastic  = r2_score(                 y_test0 ,elastic1_.predict(X_test0))
#EVS_elastic = explained_variance_score( y_test0 ,elastic1_.predict(X_test0))
#MAE_elastic = mean_absolute_error(      y_test0 ,elastic1_.predict(X_test0))
#
#R2_bayes    = r2_score(                 y_test0 ,bayes1_.predict(X_test0))
#EVS_bayes   = explained_variance_score( y_test0 ,bayes1_.predict(X_test0))
#MAE_bayes   = mean_absolute_error(      y_test0 ,bayes1_.predict(X_test0))
#
#R2_lasso    = r2_score(                 y_test0 ,lasso1_.predict(X_test0))
#EVS_lasso   = explained_variance_score( y_test0 ,lasso1_.predict(X_test0))
#MAE_lasso   = mean_absolute_error(      y_test0 ,lasso1_.predict(X_test0))
#
#R2_mpl      = r2_score(                 y_test0 ,mpl1_.predict(X_test0))
#EVS_mpl     = explained_variance_score( y_test0 ,mpl1_.predict(X_test0))
#MAE_mpl     = mean_absolute_error(      y_test0 ,mpl1_.predict(X_test0))

#R2_knn      = r2_score(                 y_test0 ,knn1_.predict(X_test0))
#EVS_knn     = explained_variance_score( y_test0 ,knn1_.predict(X_test0))
#MAE_knn     = mean_absolute_error(      y_test0 ,knn1_.predict(X_test0))

In [ ]:
RSQ = [R2_svm , R2_sgd , R2_mtask , R2_lars , R2_lasso , R2_ridge , R2_elastic  , R2_bayes , R2_mpl , R2_knn]
EVS = [EVS_svm, EVS_sgd, EVS_mtask, EVS_lars, EVS_lasso, EVS_ridge, EVS_elastic , EVS_bayes, EVS_mpl, EVS_knn]
MAE = [MAE_svm, MAE_sgd, MAE_mtask, MAE_lars, MAE_lasso, MAE_ridge, MAE_elastic , MAE_bayes, MAE_mpl, MAE_knn]
MOD = ['SVM'  , 'SGD'  , 'MTASK'  , 'LARS'  , 'Lasso'  , 'Ridge'  ,'Elastic Net', 'Bayes'  , 'MLP'  , 'KNN']
BHY = [ svm0.best_params_ , sgd0.best_params_, mtask0.best_params_, lars0.best_params_, lasso0.best_params_,
        ridge0.best_params_, elastic0.best_params_, bayes0.best_params_, mpl0.best_params_, knn0.best_params_]

In [ ]:
resumo_rs = {'Mod' : MOD,
             'R2'  : RSQ,
             'EVS' : EVS,
             'MAE' : MAE,
             'BHY' : BHY }

In [ ]:
pd_resumo_rlv0 = pd.DataFrame(data=resumo_rs)
pd_resumo_rlv0

In [ ]:
pd_resumo_rlv0.to_csv('pd_resumo_rlv0.csv')

<a name="halving"></a>

# 4) Halving Random Serach Tunning

- 4.1 [Suport Vector Machine](#svm2)
- 4.2 [Stochastic Gradient Descent](#sgd2)
- 4.3 [Multi-Task Elastic-Net](#elastic2)
- 4.4 [Least Angle Regression Model (LAR)](#LAR2)
- 4.5 [Ridge Regression](#ridge2)
- 4.6 [ElasticNet Regression](#ElasNet2)
- 4.7 [Bayesian Ridge](#bayesian2)
- 4.8 [Lasso Regression](#lasso2)
- 4.9 [Multilayer Perceptron Regression](#mlp2)
- 4.10 [Nearest Neighbors Regression](#nnr2)
- 4.11 [Resumo dos resultados](#resumo_rs2)

<a name="svm2"></a>

## 4.1) Suport Vector Machine

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Halving Random Serach](#halving)

In [ ]:
%%time
from sklearn.svm import SVR

# define search space
space = dict()
space = dict()
space['kernel']    = ['linear','poly','rbf','sigmoid']
space['gamma']     = ['scale','auto']
space['coef0']     = [row/50 for row in range(1,100,4)]
space['C']         = [row/50 for row in range(1,100,4)]
space['epsilon']   = [row/25 for row in range(5,25)]
space['degree']    = [row/500 for row in range(1,250,5)]
space['tol']       = [row/2000 for row in range(1,200,10)]
space['shrinking'] = [True, False] 

svm  = SVR()

svm1 = HalvingRandomSearchCV(svm, space, n_candidates=300, random_state=666, cv=3, 
                             max_resources=600, factor=3, n_jobs=20, scoring='f1')

svm1.fit(X_train, y_train)
#4min 3s

In [ ]:
# summarize result
print('Best Score: %s' % svm1.best_score_)
print('Best Hyperparameters: %s' % svm1.best_params_)

In [ ]:
%%time
svm1= SVR(C= 3.2633, coef0 = 0.8874, degree= 1.6077, epsilon= 2.21599, shrinking=False, tol= 0.00825) 
svm1.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error
print("R2:", r2_score(                 y_test0, svm1.predict(X_test0)))
print("EVS", explained_variance_score( y_test0, svm1.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0, svm1.predict(X_test0)))

In [ ]:
# Save to file in the current working directory
with open("pickle/svm1.pkl", 'wb') as file:
    pickle.dump(svm1, file)

<a name="sgd2"></a>

## 4.2) Stochastic Gradient Descent 

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [ ]:
%%time
from sklearn.linear_model import SGDRegressor

# define search space
space = dict()
space['alpha']          = [row/50 for row in range(40,500,10)]
space['l1_ratio']       = [row/400 for row in range(20,100,8)] 
space['loss']           = ['squared_loss']
space['penalty']        = ['l2']
space['learning_rate']  = ['invscaling']
space['epsilon']        = [row/700 for row in range(10,200,10)]
space['eta0']           = [row/1200 for row in range(1,200,10)]
space['power_t']        = [row/600 for row in range(60,300,10)]
space['early_stopping'] = [False]
space['tol']            = [row/5000 for row in range(1,60,20)]

sgd = SGDRegressor(max_iter=1500)

sgd1 = HalvingRandomSearchCV(sgd, space, n_candidates=300, random_state=666, cv=3, max_resources=1000, 
                             factor=3, n_jobs=20, scoring='f1')


sgd1.fit(X_train0, y_train0.ravel())

In [ ]:
# summarize result
print('Best Score: %s' % sgd1.best_score_)
print('Best Hyperparameters: %s' % sgd1.best_params_)

In [ ]:
%%time
svm1= SVR(C= 3.2633, coef0 = 0.8874, degree= 1.6077, epsilon= 2.21599, shrinking=False, tol= 0.00825) 
svm1.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test0, sgd1.predict(X_test0)))
print("EVS", explained_variance_score( y_test0, sgd1.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0, sgd1.predict(X_test0)))

In [ ]:
# Save to file in the current working directory
with open("pickle/sgd1.pkl", 'wb') as file:
    pickle.dump(sgd1, file)

<a name="elastic2"></a>

## 4.3) Multi-Task Elastic-Net

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [ ]:
%%time
from sklearn import linear_model

space = dict()
space['alpha']         = [row/200 for row in range(1,1200,60)]
space['l1_ratio']      = [row/30 for row in range(1,300,10)]
space['fit_intercept'] = [False,True]
space['normalize']     = [False,True]
space['warm_start']    = [False,True]
space['copy_X']        = [False,True]
space['selection']     = ['cyclic','random']
space['tol']           = [row/5000 for row in range(1,20,2)]


mtask = linear_model.MultiTaskElasticNet(max_iter=800)

mtask1 = HalvingRandomSearchCV(mtask, space, n_candidates=300, random_state=666, cv=3, max_resources=1000, 
                             factor=3, n_jobs=20, scoring='f1')

mtask1.fit(X_train, y_train)

In [ ]:
# summarize result
print('Best Score: %s' % mtask1.best_score_)
print('Best Hyperparameters: %s' % mtask1.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test,mtask1.predict(X_test)))
print("EVS", explained_variance_score( y_test,mtask1.predict(X_test)))
print("MAE", mean_absolute_error(      y_test,mtask1.predict(X_test)))

In [ ]:
# Save to file in the current working directory
with open("pickle/mtask1.pkl", 'wb') as file:
    pickle.dump(mtask1, file)

<a name="LAR2"></a>

## 4.4) Least Angle Regression Model (LAR)

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [ ]:
linear_model.LassoLars().get_params().keys()

In [ ]:
%%time
from sklearn import linear_model

space = dict()
space['alpha']          = [row/300 for row in range(1,900,10)]
space['fit_intercept']  = [False,True]
space['fit_path']       = [False,True]
space['normalize']      = [False,True]
space['copy_X']         = [False,True]
space['positive']       = [False,True]
space['eps']            = [row/100 for row in range(1000,10000,100)]


lars = linear_model.LassoLars()

lars1 = HalvingRandomSearchCV(mtask, space, n_candidates=300, random_state=666, cv=3, max_resources=1000, 
                             factor=3, n_jobs=20, scoring='f1')

lars1.fit(X_train, y_train)

In [ ]:
# summarize result
print('Best Score: %s' % lars1.best_score_)
print('Best Hyperparameters: %s' % lars1.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test,lars1.predict(X_test)))
print("EVS", explained_variance_score( y_test,lars1.predict(X_test)))
print("MAE", mean_absolute_error(      y_test,lars1.predict(X_test)))

In [ ]:
# Save to file in the current working directory
with open("pickle/lars1.pkl", 'wb') as file:
    pickle.dump(lars1, file)

<a name="ridge2"></a>

## 4.5) Ridge Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [ ]:
%%time
from sklearn.linear_model import Ridge
space = dict()
space['alpha']         = [row/500 for row in range(8,100,5)]
space['copy_X']        = [False,True]
space['fit_intercept'] = [False,True]
space['normalize']     = [False,True]
space['solver']        = ['auto','svd','cholesky','lsqr','sparse_cg']
space['tol']           = [row/100000 for row in range(1,1000,10)]

ridge = Ridge()

ridge1 = HalvingRandomSearchCV(ridge, space, n_candidates=300, random_state=666, cv=3, factor=3, n_jobs=20, scoring='f1')

ridge1.fit(X_train, y_train)

In [ ]:
# summarize result
print('Best Score: %s' % ridge1.best_score_)
print('Best Hyperparameters: %s' % ridge1.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test, ridge1.predict(X_test)))
print("EVS", explained_variance_score( y_test, ridge1.predict(X_test)))
print("MAE", mean_absolute_error(      y_test, ridge1.predict(X_test)))

In [ ]:
# Save to file in the current working directory
with open("pickle/ridge1.pkl", 'wb') as file:
    pickle.dump(ridge1, file)

<a name="ElasNet2"></a>

## 4.6) ElasticNet Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)



In [ ]:
%%time
from sklearn.linear_model import ElasticNet

space = dict()
space['alpha']         = [row/500 for row in range(1,500,5)]
space['l1_ratio']      = [row/250 for row in range(5,1000,10)]
space['fit_intercept'] = [False,True]
space['normalize']     = [False,True]
space['precompute']    = [False,True]
space['copy_X']        = [False,True]
space['warm_start']    = [False,True]
space['positive']      = [False,True]
space['selection']     = ['cyclic','random']
space['tol']           = [row/10000 for row in range(10,10000,10)]

elastic = ElasticNet()

elastic1 = HalvingRandomSearchCV(elastic, space, n_candidates=300, random_state=666, cv=3, factor=3, n_jobs=20, scoring='f1')

elastic1.fit(X_train, y_train)

In [ ]:
# summarize result
print('Best Score: %s' % elastic1.best_score_)
print('Best Hyperparameters: %s' % elastic1.best_params_)

In [ ]:
# summarize result
print('Best Score: %s' % elastic1.best_score_)
print('Best Hyperparameters: %s' % elastic1.best_params_)

In [ ]:
# Save to file in the current working directory
with open("pickle/elastic1.pkl", 'wb') as file:
    pickle.dump(elastic1, file)

In [85]:
# Save to file in the current working directory
with open("pickle/elastic1.pkl", 'wb') as file:
    pickle.load(elastic1, file)

NameError: name 'elastic1' is not defined

<a name="bayesian2"></a>

## 4.7) Bayesian Ridge Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)


In [ ]:
%%time   
from sklearn.linear_model import BayesianRidge

space = dict()
space['alpha_1']        = [row/40000000 for row in range(1,2000,100)]
space['alpha_2']        = [row/40000000 for row in range(1,2000,100)]
space['lambda_1']       = [row/40000000 for row in range(1,2000,100)]
space['lambda_2']       = [row/40000000 for row in range(1,2000,100)]
space['compute_score']  = [False]
space['copy_X']         = [True]
space['fit_intercept']  = [True]
space['normalize']      = [False]
space['tol']            = [row/5000 for row in range(1,100,10)]

bayes = BayesianRidge()

bayes1 = HalvingRandomSearchCV(bayes, space, n_candidates = 300 , max_resources=50, 
                               random_state=666, cv=3, factor=3, n_jobs=20 , scoring='f1')
bayes1.fit(X_train, y_train.values.ravel())

In [ ]:
# summarize result
print('Best Score: %s' % bayes1.best_score_)
print('Best Hyperparameters: %s' % bayes1.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test, bayes1.predict(X_test)))
print("EVS", explained_variance_score( y_test, bayes1.predict(X_test)))
print("MAE", mean_absolute_error(      y_test, bayes1.predict(X_test)))

In [ ]:
# Save to file in the current working directory
with open("pickle/bayes1.pkl", 'wb') as file:
    pickle.dump(bayes1, file)

<a name="lasso2"></a>

## 4.8) Lasso Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)



In [ ]:
%%time
from sklearn.linear_model import Lasso

space = dict()
space['alpha']         = [row/1000 for row in range(10,100,5)]
space['copy_X']        = [False,True]
space['fit_intercept'] = [False,True]
space['normalize']     = [False,True]
space['precompute']    = [False,True]
space['positive']      = [False,True]
space['selection']     = ['cyclic','random']
space['tol']           = [row/1000000 for row in range(100,10000,10)]


lasso = Lasso()

lasso1 = HalvingRandomSearchCV(lasso, space, n_candidates = 300 , max_resources=50, 
                               random_state=666, cv=3, factor=3, n_jobs=20 , scoring='f1')

lasso1.fit(X_train, y_train.values.ravel())

In [ ]:
# summarize result
print('Best Score: %s' % lasso1.best_score_)
print('Best Hyperparameters: %s' % lasso1.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test, lasso1.predict(X_test)))
print("EVS", explained_variance_score( y_test, lasso1.predict(X_test)))
print("MAE", mean_absolute_error(      y_test, lasso1.predict(X_test)))

In [ ]:
# Save to file in the current working directory
with open("pickle/lasso1.pkl", 'wb') as file:
    pickle.dump(lasso1, file)

<a name="mlp2"></a>

## 4.9) Multilayer Perceptron

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)



In [ ]:
%%time
from sklearn.neural_network import MLPRegressor

space = dict()
space['hidden_layer_sizes']  = [(100,50), (25,100,25), (75,50,75)]
space['activation']          = ['identity','logistic','tanh','relu']
space['solver']              = ['lbfgs', 'sgd', 'adam']
space['alpha']               = [row/200 for row in range(1,100,20)]
space['learning_rate']       = ['constant','invscaling','adaptive']
space['learning_rate_init']  = [row/1000 for row in range(1,20,10)]
space['warm_start']          = [False]
space['power_t']             = [row/50 for row in range(1,200,40)]
space['tol']                 = [row/50000000 for row in range(1,1000,400)]
space['momentum']            = [row/200 for row in range(1,100,20)]
space['nesterovs_momentum']  = [True]
space['early_stopping']      = [False]
space['beta_1']              = [row/200 for row in range(1,20,12)]
space['beta_2']              = [row/200 for row in range(1,20,12)]
space['epsilon']             = [row/200 for row in range(1,20,12)]
    
mpl = MLPRegressor(max_iter=1200)

mpl1 = HalvingRandomSearchCV(mpl, space, n_candidates = 300 , max_resources=50, 
                               random_state=666, cv=3, factor=3, n_jobs=20 , scoring='f1')

mpl1.fit(X_train0, y_train0.ravel())

In [ ]:
# summarize result
print('Best Score: %s' % mpl1.best_score_)
print('Best Hyperparameters: %s' % mpl1.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test0, mpl1.predict(X_test0)))
print("EVS", explained_variance_score( y_test0, mpl1.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0, mpl1.predict(X_test0)))

In [ ]:
# Save to file in the current working directory
with open("pickle/mpl1.pkl", 'wb') as file:
    pickle.dump(mpl1, file)

<a name="nnr2"></a>

## 4.10) Nearest Neighbors Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)


In [ ]:
%%time
from sklearn.neighbors import KNeighborsRegressor

space = dict()
space['n_neighbors'] = [row for row in range(4,10,2)]
space['weights']     = ['uniform', 'distance']
space['algorithm']   = ['auto', 'ball_tree', 'kd_tree', 'brute']
space['leaf_size']   = [row for row in range(10,50,2)]
space['p']           = [row for row in range(2,8,2)]
    
knn = KNeighborsRegressor()

knn1 = HalvingRandomSearchCV(knn, space, n_candidates = 300 , max_resources=50, 
                               random_state=666, cv=3, factor=3, n_jobs=20 , scoring='f1')

knn1.fit(X_train, y_train)

In [ ]:
# summarize result
print('Best Score: %s' % knn1.best_score_)
print('Best Hyperparameters: %s' % knn1.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test, knn1.predict(X_test)))
print("EVS", explained_variance_score( y_test, knn1.predict(X_test)))
print("MAE", mean_absolute_error(      y_test, knn1.predict(X_test)))

In [ ]:
# Save to file in the current working directory
with open("pickle/knn1.pkl", 'wb') as file:
    pickle.dump(knn1, file)

<a name="resumo_rs2"></a>

## 4.11) Resumo dos resultados

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)


In [ ]:
R2_mpl      = r2_score(                 y_test0, mpl1.predict(X_test0))
EVS_mpl     = explained_variance_score( y_test0, mpl1.predict(X_test0))
MAE_mpl     = mean_absolute_error(      y_test0, mpl1.predict(X_test0))

R2_lasso    = r2_score(                 y_test0, lasso1.predict(X_test0))
EVS_lasso   = explained_variance_score( y_test0, lasso1.predict(X_test0))
MAE_lasso   = mean_absolute_error(      y_test0, lasso1.predict(X_test0))

R2_bayes    = r2_score(                 y_test0, bayes1.predict(X_test0))
EVS_bayes   = explained_variance_score( y_test0, bayes1.predict(X_test0))
MAE_bayes   = mean_absolute_error(      y_test0, bayes1.predict(X_test0))

R2_elastic  = r2_score(                 y_test0, elastic1.predict(X_test0))
EVS_elastic = explained_variance_score( y_test0, elastic1.predict(X_test0))
MAE_elastic = mean_absolute_error(      y_test0, elastic1.predict(X_test0))

R2_ridge    = r2_score(                 y_test , ridge1.predict(X_test))
EVS_ridge   = explained_variance_score( y_test , ridge1.predict(X_test))
MAE_ridge   = mean_absolute_error(      y_test , ridge1.predict(X_test))

R2_lars     = r2_score(                 y_test ,lars1.predict(X_test))
EVS_lars    = explained_variance_score( y_test ,lars1.predict(X_test))
MAE_lars    = mean_absolute_error(      y_test ,lars1.predict(X_test))

R2_sgd      = r2_score(                 y_test ,sgd1.predict(X_test))
EVS_sgd     = explained_variance_score( y_test ,sgd1.predict(X_test))
MAE_sgd     = mean_absolute_error(      y_test ,sgd1.predict(X_test))

R2_svm      = r2_score(                 y_test0 ,svm1.predict(X_test0))
EVS_svm     = explained_variance_score( y_test0 ,svm1.predict(X_test0))
MAE_svm     = mean_absolute_error(      y_test0 ,svm1.predict(X_test0))

R2_knn      = r2_score(                 y_test0, knn1.predict(X_test0))
EVS_knn     = explained_variance_score( y_test0, knn1.predict(X_test0))
MAE_knn     = mean_absolute_error(      y_test0, knn1.predict(X_test0))

In [ ]:
RSQ = [R2_mpl , R2_lasso , R2_bayes , R2_elastic   , R2_ridge  , R2_lars  , R2_sgd , R2_svm , R2_knn]
EVS = [EVS_mpl, EVS_lasso, EVS_bayes, EVS_elastic  , EVS_ridge , EVS_lars , EVS_sgd, EVS_svm, EVS_knn]
MAE = [MAE_mpl, MAE_lasso, MAE_bayes, MAE_elastic  , MAE_ridge , MAE_lars , MAE_sgd, MAE_svm, MAE_knn]
MOD = ['MLP'  , 'Lasso'  , 'Bayes'  ,'Elastic Net' , 'Ridge'   , 'LARS'   , 'SGD'  ,'SVM Reg', 'KNN']
BHY = [ mpl1.best_params_ ,lasso1.best_params_  ,bayes1.best_params_  ,elastic1.best_params_ ,
        ridge1.best_params_ ,lars1.best_params_ ,sgd1.best_params_  ,svm1.best_params_ , knn1.best_params_]

In [ ]:
resumo_gs = {'Mod' : MOD, 
             'R2'  : RSQ, 
             'EVS' : EVS, 
             'MAE' : MAE,
             'BHY' : BHY}

In [ ]:
pd_resumo_rlv1 = pd.DataFrame(data=resumo_gs)
pd_resumo_rlv1

In [ ]:
pd_resumo_rlv1.to_csv('pd_resumo_rlv1.csv')

In [ ]:
import pickle
xgboost   = pickle.load(open('pickle/xgboost1_.pkl', 'rb'))
sgd       = pickle.load(open('pickle/sgd1_.pkl', 'rb'))
mtask     = pickle.load(open('pickle/mtask1_.pkl', 'rb'))
lars      = pickle.load(open('pickle/lars1_.pkl', 'rb'))
ridge     = pickle.load(open('pickle/ridge1_.pkl', 'rb'))    
elastic   = pickle.load(open('pickle/elastic1_.pkl', 'rb'))
bayes     = pickle.load(open('pickle/bayes1_.pkl', 'rb'))
lasso     = pickle.load(open('pickle/lasso1_.pkl', 'rb'))
mpl       = pickle.load(open('pickle/mpl1_.pkl', 'rb'))
#knn       = pickle.load(open('pickle/knn1_.pkl', 'rb'))

In [125]:
feat_xgboost=pd.DataFrame(xgboost.feature_importances_, columns=['feat_xgboost']) 
feat_xgboost['features'] = lista

feat_sgd=pd.DataFrame(sgd.coef_, columns=['feat_sgd']) 
feat_sgd['features'] = lista

feat_mtask=pd.DataFrame(mtask.coef_[0], columns=['feat_mtask']) 
feat_mtask['features'] = lista

feat_lars=pd.DataFrame(lars.coef_[0], columns=['feat_lars']) 
feat_lars['features'] = lista

feat_ridge=pd.DataFrame(ridge.coef_[0], columns=['feat_ridge']) 
feat_ridge['features'] = lista

In [126]:
feat_xgboost.set_index('features',inplace=True)
feat_sgd.set_index('features',inplace=True)
feat_mtask.set_index('features',inplace=True)
feat_lars.set_index('features',inplace=True)
feat_ridge.set_index('features',inplace=True)

In [129]:
frames = [feat_xgboost, feat_sgd, feat_mtask, feat_lars, feat_ridge]
t=pd.concat(frames,axis=1)

In [131]:
t.head()

,feat_xgboost,feat_sgd,feat_mtask,feat_lars,feat_ridge
features,,,,,
banco_ativo,0.000917,-0.001152,0.032212,0.0,0.121141
banco_ativo_captacoes,0.000960,-0.000201,0.017924,0.0,-3.781824
banco_ativo_carteira,0.001888,-0.000215,0.168003,0.0,2.918747
banco_ativo_passivo,0.000896,-0.000199,0.000000,0.0,-0.312344
banco_ativo_patrimonio,0.000740,-0.000214,0.085080,0.0,3.861496


In [134]:
t.to_excel("features.xlsx", sheet_name='features')